# Assumptions:
### --- Assume all blank columns and length of 1; assume "0.2x size" format wont change; assume that one tradetype has only one number for size calculating (such as "10", "5")
# Do it manually
### make sure for each CCY there is only ONE blank column between each "small table", if more than 1, delete the rest

In [111]:
import pandas as pd
import numpy as np
import xlrd
sheets_dict = pd.read_excel("Input_File.xlsx", sheet_name=None)
workbook = xlrd.open_workbook("Input_File.xlsx")

In [112]:
def check_not_empty(worksheet,row,col):
        return worksheet.cell(row,col).value != ''

def check_whole_col_empty(worksheet,tot_rows,col):
    result = True
    for i in range(tot_rows):
        if check_not_empty(worksheet,i,col):
            result = False
            break;
    return result;

def check_whole_row_empty(worksheet,row,total_cols):
    result = True
    for i in range(total_cols):
        if check_not_empty(worksheet,row,i):
            result = False
            break;
    return result;

def combine_consecutive_number(alist):
    seat_need_delete = []
    for i in range(len(alist)-1):
        if alist[i+1] == alist[i]+1:
            seat_need_delete.append(alist[i])
    result = list(set(alist)-set(seat_need_delete))
    result.sort()
    return result

def str_contains(str1,str2,str3='yuezhou',str4='yuezhou'):
    result = False
    if str2 in str1:
        result = True
    if str3 in str1:
        result = True
    if str4 in str1:
        result = True
    return result;

def get_size_from_string(astring):
    astring = astring.replace('x','')
    
    return float(astring.split()[0])

def get_market_survey(worksheet,rows,cols):
    market_survey = []
    arrayofvalues = worksheet.col_values(0)
    last_row = len(arrayofvalues)
    first_row = rows[-1]
    last_col = cols[-1]
    for i in range(rows[-1],last_row):
        row_curr = i
        total_cols = last_col
        if not check_whole_row_empty(worksheet,row_curr,total_cols-1):
            row =  worksheet.row_values(row_curr)
            market_survey.append(row)
    
    cols_need = cols[1:-1]
    cols_need = [x+1 for x in cols_need]
    cols_to_add = list(range(cols_need[0]))
    cols_need = cols_to_add + cols_need
    return pd.DataFrame(data = market_survey).iloc[:,cols_need]

def divide_bycompany(df,company_list):
    return [df[df['Firm']==co] for co in company_list]

def right_join(pack):
    df_right = pack[0].copy().drop(['Spread'], axis=1)
    i=0
    for df2 in pack:
        name = firms[i]
        i=i+1        
        #df_right[name] = df2.Spread.reset_index().Spread
        df_right.insert(i+4, name, df2.Spread.tolist() , True) 
    return df_right

In [113]:
sheet_names = workbook.sheet_names()
worksheets = {}
for i in range(len(sheet_names)-1):
    worksheets[sheet_names[i+1]] = workbook.sheet_by_name(sheet_names[i+1])

### Get all blank rows and cols

In [114]:
for ith_worksheet in range(len(sheet_names)-1):
    worksheet_now = worksheets[sheet_names[1+ith_worksheet]]
    n_cols = worksheet_now.ncols
    n_rows = worksheet_now.nrows

    effective_cols =[]
    effective_rows = []

    for i in range(n_cols):
        if not check_whole_col_empty(worksheet_now,n_rows,i):
            effective_cols.append(i)
        
    empty_cols = list(set(range(0,n_cols))-set(effective_cols))
    empty_cols.sort()        

    # attention: first 2 in the effective cols are "median" and "bis/ask spread for a trade"
    for i in range(n_rows):
        if check_not_empty(worksheet_now,i,empty_cols[0]+1):
            effective_rows.append(i)
        
    above_table_empty_rows = list(set(range(effective_rows[0],effective_rows[-1]))-set(effective_rows))
    above_table_empty_rows.sort()
    above_table_empty_rows = combine_consecutive_number(above_table_empty_rows)

    ### get CCY --list
    currency = worksheet_now.cell(4,0).value
    ### Get Trade Type --list
    tradetype = []
    for i in above_table_empty_rows:
        # no need to plus 1 here
        tradetype.append(worksheet_now.cell(i+1,0).value)
    ### Get Firm name --list
    firms = []
    for i in range(n_cols):
        if check_not_empty(worksheet_now,above_table_empty_rows[0]-1,i):
            firms.append(worksheet_now.cell(above_table_empty_rows[0]-1,i).value)
        
    ###  tenors (and sizes) -- a dict  indexing on Trade Type --- and is same for each company
    # different trade type may have different ternor collection
    tenors = {}
    size_bases = {}
    frame_rows = above_table_empty_rows.copy()
    frame_cols = empty_cols.copy()
    frame_rows.append(effective_rows[-1]+1)
    frame_cols.append(effective_cols[-1]+1)
    j=0
    # first loop: each descriptive table:higher to lower

    for i in range(len(frame_rows)-1): 
    
        symbol = '+'
        row_inuse = frame_rows[i]
        next_row = frame_rows[i+1]
        tenor_ele = {}
        tenor_choice = []
        size_choice = []
    
        name = tradetype[j]
    
        for rows_in_table in range(row_inuse+2,next_row):
            a_tenor = ''
            a_size = 0
            temp_pair = []
            if rows_in_table == row_inuse+2:
                for k in range(1,empty_cols[0]):
                    if check_not_empty(worksheet_now,rows_in_table,k):
                        tenor_ele[k] = worksheet_now.cell(rows_in_table,k).value
                    
            if rows_in_table != row_inuse+2:    
                for k in range(1,empty_cols[0]): # empty is in fact the furtherest 
                    if check_not_empty(worksheet_now,rows_in_table,k):
                        if  worksheet_now.cell(rows_in_table,k).value >0:
                            symbol = '+'
                        else:
                            symbol = '-'
                        #    ca_tenor= a_tenor + symbol + tenor_ele[k]+ ' '
                        a_tenor= a_tenor  + tenor_ele[k]+ '_'
                        a_size = worksheet_now.cell(rows_in_table,k).value
                        temp_pair.append(a_tenor)
                        temp_pair.append((a_size)*1000000)
                        a_size = temp_pair
                tenor_choice.append(a_tenor[:-1])
                #size_choice.append(abs(a_size)*1000000)
                size_choice.append((a_size))
        tenors[name] = tenor_choice  
        size_bases[name] = size_choice
        #size_bases[name] = temp_pair
        j=j+1

    ### size multiplier; which indexing on tradetype
    size_mul = {}
    for i in above_table_empty_rows:
        multiplier = []
        name = worksheet_now.cell(i+1,0).value
        for j in range(empty_cols[0]+1,empty_cols[1]):
            if check_not_empty(worksheet_now,i+2,j):
                multiplier.append(get_size_from_string(worksheet_now.cell(i+2,j).value))
                #multiplier.append(worksheet_now.cell(i+2,j).value)
            else:pass;
        size_mul[name] = multiplier

    ### loop each company to find out the spread
    output = pd.DataFrame(columns= ['CCY','Trade_Type','Firm','Tenor','DV01_Size','Bid_Ask_Spread'])
    spreads = []
    for j in range(len(frame_rows)-1):    
        row_inuse = frame_rows[j]
        #print(row_inuse)
        row_next = frame_rows[j+1]

        for i in range(len(frame_cols)-1):
            col_inuse = frame_cols[i]+1
            col_next = frame_cols[i+1]
            for arow in range(row_inuse+3,row_next):
                for acol in range(col_inuse,col_next):
                    spreads.append(worksheet_now.cell(arow,acol).value)
                    #print(arow,acol)
                
    ### prepare sizes
    sizes = {}
    for name in tradetype:
        l1 = size_bases[name]
        l2 = size_mul[name]
        temp_lista = []
        temp_listb = []
        for i in range(len(l1)):
            a_tenor = tenors[name][i]
            for j in range(len(l2)):
                _num = l1[i][1] # it is the number of base
                _mul = l2[j]
                temp_lista.append([_num*_mul,a_tenor])
        sizes[name] = temp_lista
    
    #for ccy in sheet_names:
    test=0
    classify = []
    temp_num = 0
    for tt in tradetype:
        for firm in firms:
                for siz in sizes[tt]:
                    
                    #
                    
                    li = [worksheet_now.name,tt,firm,siz[0],siz[1],spreads[temp_num]]
                    temp_num = temp_num+1
                    classify.append(li)

    df_output = pd.DataFrame.from_records(classify)
    df_output.columns=['CCY','TradeType','Firm','Size','Tenor','Spread']
    filename = 'output_' + sheet_names[1+ith_worksheet]+'.csv'
    df_output.to_csv(filename,index=False)
    
    market_survey = get_market_survey(worksheet_now,frame_rows,frame_cols)
    filename = 'market_survey_' + sheet_names[1+ith_worksheet]+'.csv'
    market_survey.to_csv(filename,index=False)
    
    by_company = right_join(divide_bycompany(df_output,firms))
    filename = 'By_company_' + sheet_names[1+ith_worksheet]+'.csv'
    by_company.to_csv(filename,index=False)

In [149]:
df_operate = by_company.iloc[:,6:29]
df_operate = df_operate.reset_index()
index_ = df_operate.index
df_operate = df_operate.T
list_stat = []
list_stats = []
for i in index_:
    list_stat.append(np.nanmean(df_operate.iloc[1:,i].replace( '', np.nan, regex=True)))

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [153]:
df_operate.iloc[1:,i].replace( '', np.nan, regex=True).tolist()

[25.0,
 nan,
 nan,
 4.050000000000001,
 nan,
 nan,
 nan,
 nan,
 nan,
 150.0,
 nan,
 nan,
 nan,
 nan,
 '\xa025',
 25.0,
 32.0,
 40.0,
 500.0,
 nan,
 nan,
 nan,
 190.0]

In [158]:
df_operate.iloc[1:,i]

TypeError: 'numpy.ndarray' object is not callable

In [159]:
df_operate

,0,1,2,3,4,5,6,7,8,9,...,198,199,200,201,202,203,204,205,206,207
index,0,1,2,3,4,5,6,7,8,9,...,4246,4247,4248,4249,4250,4251,4252,4253,4254,4255
DB,4,7,10,13,16,20,25,30,4,7,...,15,20,2,2,3,4,6,10,15,20
BNP,15,30,60,120,170,220,,,15,30,...,,,5,5,10,20,30,60,,
RABO,,,,,,,,,,,...,,,,,,,,,,
Nomura,5,7,10,18,25,30,40.5,54.675,5,7,...,21.5,28,7,7,7,7.5,14,17.5,24.75,32
UBS,,,,,,,,,,,...,,,,,,,,,,
BMO,,,,,,,,,,,...,,,,,,,,,,
TD,,,,,,,,,,,...,,,,,,,,,,
Credit Suisse,10,13,15,20,25,40,,,12,15,...,,,5,8,10,12,15,20,,
HSBC,10,15,30,50,75,125,,,10,15,...,,,2.41577,3.62365,7.2473,14.4946,24.1577,41,,


In [160]:
worksheet_now.name

'CLP'